In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///bank_marketing.db

In [ ]:
%%sql
CREATE TABLE bank_marketing (
    age INT,
    job VARCHAR(50),
    marital VARCHAR(50),
    education VARCHAR(50),
    default_status VARCHAR(10),
    balance INT,
    housing VARCHAR(10),
    loan VARCHAR(10),
    contact VARCHAR(50),
    day INT,
    month VARCHAR(10),
    duration INT,
    campaign INT,
    pdays INT,
    previous INT,
    poutcome VARCHAR(50),
    y VARCHAR(10)
);

 * sqlite:///bank_marketing.db
Done.


[]

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
import pandas as pd

# Read the CSV file with the correct delimiter and quoting
df = pd.read_csv('train.csv', delimiter=';', quotechar='"')

# Rename the column to avoid conflict with SQL reserved keyword
df.rename(columns={'default': 'default_status'}, inplace=True)

## **Data Preprocessing and Cleaning**

In [ ]:
# Step 1: Check for missing values
missing_values = df.isnull().sum()

# Step 2: Standardize categorical variables to lowercase
categorical_columns = ['job', 'marital', 'education', 'default_status', 'housing', 'loan', 'contact', 'poutcome', 'month', 'y']
df[categorical_columns] = df[categorical_columns].apply(lambda x: x.str.lower())

# Step 3: Replace 'unknown' with NaN for categorical columns
df.replace('unknown', pd.NA, inplace=True)

# Fill NaNs in categorical columns with mode
for col in categorical_columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Fill NaNs in numeric columns with median
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_columns:
    df[col].fillna(df[col].median(), inplace=True)

# Step 4: Detect and treat outliers in numeric columns (e.g., balance)
df['balance'] = df['balance'].clip(lower=df['balance'].quantile(0.01), upper=df['balance'].quantile(0.99))

# Display the first few rows of the cleaned dataframe
df.head()

,age,job,marital,education,default_status,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,cellular,5,may,261,1,-1,0,failure,no
1,44,technician,single,secondary,no,29.0,yes,no,cellular,5,may,151,1,-1,0,failure,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,cellular,5,may,76,1,-1,0,failure,no
3,47,blue-collar,married,secondary,no,1506.0,yes,no,cellular,5,may,92,1,-1,0,failure,no
4,33,blue-collar,single,secondary,no,1.0,no,no,cellular,5,may,198,1,-1,0,failure,no


In [ ]:
from sqlalchemy import create_engine

# Create a connection to the SQLite database
engine = create_engine('sqlite:///bank_marketing.db')

# Load the data into the SQL table
df.to_sql('bank_marketing', con=engine, if_exists='append', index=False)


45211

In [ ]:
# Verify the data upload
%sql sqlite:///bank_marketing.db

In [ ]:
# Check the first few rows of the table
%%sql
SELECT * FROM bank_marketing LIMIT 5;

 * sqlite:///bank_marketing.db
Done.


age,job,marital,education,default_status,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
58,management,married,tertiary,no,2143,yes,no,cellular,5,may,261,1,-1,0,failure,no
44,technician,single,secondary,no,29,yes,no,cellular,5,may,151,1,-1,0,failure,no
33,entrepreneur,married,secondary,no,2,yes,yes,cellular,5,may,76,1,-1,0,failure,no
47,blue-collar,married,secondary,no,1506,yes,no,cellular,5,may,92,1,-1,0,failure,no
33,blue-collar,single,secondary,no,1,no,no,cellular,5,may,198,1,-1,0,failure,no


## **Campaign Effectiveness**

In [ ]:
%%sql
SELECT campaign, COUNT(*) AS total_contacts,
       SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS successful_contacts
FROM bank_marketing
GROUP BY campaign;


 * sqlite:///bank_marketing.db
Done.


campaign,total_contacts,successful_contacts
1,17544,2561
2,12505,1401
3,5521,618
4,3522,317
5,1764,139
6,1291,92
7,735,47
8,540,32
9,327,21
10,266,14


Insights:

**1.   Higher Campaign Efforts Yield More Success**: Campaigns with a higher number of total contacts generally show a higher number of successful contacts. For instance, Campaign 1 with the most contacts (17,544) has the highest number of successful contacts (2,561).

**2.   Diminishing Returns**: As the number of contacts decreases, the number of successful contacts also decreases, but not proportionally. For example, Campaign 2 with 12,505 contacts yields 1,401 successes, which is significantly lower in proportion compared to Campaign 1.

**3.   Low Effectiveness in Small Campaigns**: Campaigns with fewer contacts (e.g., Campaigns with 10 or fewer contacts) generally have zero or very few successful contacts, indicating that small-scale campaigns are less effective.

## **Customer Segmentation**

In [ ]:
%%sql
SELECT job, COUNT(*) AS total_customers,
       SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS successful_subscriptions
FROM bank_marketing
GROUP BY job;

 * sqlite:///bank_marketing.db
Done.


job,total_customers,successful_subscriptions
admin.,5171,631
blue-collar,10020,742
entrepreneur,1487,123
housemaid,1240,109
management,9458,1301
retired,2264,516
self-employed,1579,187
services,4154,369
student,938,269
technician,7597,840


Insights:

**1.   High Success in Management**: The 'management' job category has the highest number of successful subscriptions (1,301) despite having a slightly lower total number of customers compared to 'blue-collar'.

**2.   Retirees and Students Are More Receptive**: The 'retired' and 'student' categories have relatively high success rates, indicating these groups might be more receptive to marketing efforts.

## **Subscription Rate by Education Level**

In [ ]:
%%sql
SELECT education, COUNT(*) AS total_customers,
       SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS successful_subscriptions,
       ROUND(SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS subscription_rate
FROM bank_marketing
GROUP BY education
ORDER BY subscription_rate DESC;

 * sqlite:///bank_marketing.db
Done.


education,total_customers,successful_subscriptions,subscription_rate
tertiary,13301,1996,15.01
secondary,25059,2702,10.78
primary,6851,591,8.63


## **Effectiveness by Contact Method**

In [ ]:
%%sql
SELECT contact, COUNT(*) AS total_contacts,
       SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS successful_contacts,
       ROUND(SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS success_rate
FROM bank_marketing
GROUP BY contact
ORDER BY success_rate DESC;


 * sqlite:///bank_marketing.db
Done.


contact,total_contacts,successful_contacts,success_rate
telephone,2906,390,13.42
cellular,42305,4899,11.58


## **Monthly Performance**

In [ ]:
%%sql
SELECT month, COUNT(*) AS total_contacts,
       SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) AS successful_contacts,
       ROUND(SUM(CASE WHEN y = 'yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS success_rate
FROM bank_marketing
GROUP BY month
ORDER BY success_rate DESC;


 * sqlite:///bank_marketing.db
Done.


month,total_contacts,successful_contacts,success_rate
mar,477,248,51.99
dec,214,100,46.73
sep,579,269,46.46
oct,738,323,43.77
apr,2932,577,19.68
feb,2649,441,16.65
aug,6247,688,11.01
jun,5341,546,10.22
nov,3970,403,10.15
jan,1403,142,10.12


## **Conclusion**


1. **Campaign Effectiveness**:
   - **Campaign Efforts**: Campaigns with a higher number of total contacts tend to have a higher number of successful contacts. However, diminishing returns are observed, suggesting that indefinitely increasing the number of contacts does not necessarily improve results in the same proportion. For example, Campaign 1 with 17,544 contacts had 2,561 successful contacts, while other campaigns with fewer contacts were significantly less successful.

2. **Customer Segmentation by Occupation**:
   - **High Responsiveness in Managers and Technicians**: Customers in the 'management' and 'technician' categories show high subscription rates. These groups should be prioritized in future campaigns, as they show higher receptivity to time deposits.
   - **Retirees and Students**: High receptivity was also observed in the retired and student categories, suggesting that these groups may be good targets for marketing campaigns.

3. **Subscription Rate by Educational Level**:
   - **Higher Education, Higher Subscription Rate**: Clients with tertiary education have the highest subscription rate (15.01%), followed by those with secondary (10.78%) and primary (8.63%) education. This indicates that clients with higher levels of education are more inclined to invest in time deposits.

4. **Effectiveness by Method of Contact**:
   - **Phone over Cellular**: The telephone contact method has a success rate of 13.42%, higher than contact via cellular at 11.58%. This suggests that, despite the higher cost, telephone contact may be more effective for targeted campaigns, especially with identified prospects.

5. **Monthly Performance**:
   - **Significant Variations**: Months such as March, December and September show significantly higher success rates, suggesting that there is seasonality in customer behavior. This pattern can be used to plan marketing campaigns more accurately by focusing on these months to maximize results.

#### Treatment of Unknown Values ("unknown")

In this analysis, it was decided to replace the "unknown" values with the mode of each column to ensure that the data were complete and ready for analysis. This is a basic and quick strategy for handling missing data, but it has its limitations, as it can introduce bias by skewing the distribution of the data towards the mode.

**Advanced and Correct Practices for Handling "Unknowns "**:

1. **Missing Data Causality Analysis**:
   - **MCAR (Missing Completely at Random)**: Missing data are not related to any other variable. If data are suspected to be MCAR, elimination of these records can be considered without introducing bias.
   - **Missing at Random**: Missing data are dependent on other observable variables. In this case, imputation techniques such as regression models can be used to predict missing values based on other characteristics.
   - **MNAR (Missing Not at Random)**: Missing data depend on the missing values themselves. It is crucial to identify this pattern to properly handle missing data, possibly through more advanced analysis or additional surveys.

2. **Advanced Imputation Techniques**:
   - **Multiple Imputation**: Generate multiple imputations for each missing value and average the results to reflect the uncertainty of the predictions.
   - Predictive Modeling**: Use machine learning algorithms such as K-Nearest Neighbors (KNN) or Random Forest to predict missing values based on other variables.

3. **Sensitivity Analysis**:
   - Perform a sensitivity analysis to understand how the missing data imputation affects the results of the analysis. This may include comparing results with different imputation methods or verifying results with complete and incomplete data.

### Summary

The analysis has identified key patterns and priority customer segments for future marketing campaigns. However, the management of missing data and the choice of imputation methods are critical to ensure the validity and reliability of the results. It is essential to use advanced and correct techniques to handle unknowns and other missing data, thus avoiding potential biases and ensuring that conclusions are robust and actionable.